<a href="https://colab.research.google.com/github/Manideepvadlagatta/Trader-Behavior-Insights/blob/main/Notebook_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# importing libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Creatation of submission structure
root = "ds_ManideepVadlagatta"
csv_dir = os.path.join(root, "csv_files")
out_dir = os.path.join(root, "outputs")
os.makedirs(csv_dir, exist_ok=True)
os.makedirs(out_dir, exist_ok=True)

# Loading of requried datasets into the colab
df_sent = pd.read_csv("/content/ds_ManideepVadlagatta/csv_files/fear_greed_index.csv")
df_tr = pd.read_csv("/content/ds_ManideepVadlagatta/csv_files/historical_data.csv")

#  Cleaning of  Data
df_sent.columns = df_sent.columns.str.strip().str.lower().str.replace(" ", "_")
df_sent['date'] = pd.to_datetime(df_sent['date'], errors='coerce')

# Grouping Extreme Fear -> Fear, Extreme Greed -> Greed
df_sent['classification'] = df_sent['classification'].replace({
    'Extreme Fear': 'Fear',
    'Extreme Greed': 'Greed'
})
df_tr.columns = df_tr.columns.str.strip().str.lower().str.replace(" ", "_")

# Parse date from Timestamp IST
if 'timestamp_ist' in df_tr.columns:
    df_tr['timestamp_ist'] = pd.to_datetime(df_tr['timestamp_ist'], format="%d-%m-%Y %H:%M", errors='coerce')
    df_tr['date'] = pd.to_datetime(df_tr['timestamp_ist'].dt.date)
elif 'timestamp' in df_tr.columns:
    df_tr['timestamp'] = pd.to_numeric(df_tr['timestamp'], errors='coerce')
    if df_tr['timestamp'].dropna().gt(1e12).any():
        df_tr['timestamp'] = pd.to_datetime(df_tr['timestamp'], unit='ms', errors='coerce')
    else:
        df_tr['timestamp'] = pd.to_datetime(df_tr['timestamp'], unit='s', errors='coerce')
    df_tr['date'] = pd.to_datetime(df_tr['timestamp'].dt.date)
else:
    raise ValueError("No timestamp column found for trader data.")

# Ensure Closed PnL numeric
if 'closed_pnl' in df_tr.columns:
    df_tr['closedpnl'] = pd.to_numeric(df_tr['closed_pnl'], errors='coerce')
elif 'closedpnl' in df_tr.columns:
    df_tr['closedpnl'] = pd.to_numeric(df_tr['closedpnl'], errors='coerce')
else:
    raise ValueError("No closed_pnl column found in trader data.")

#  Merge datasets
df_merged = pd.merge(df_tr, df_sent[['date', 'classification']], on="date", how="left")
df_merged.to_csv(os.path.join(csv_dir, "merged_dataset.csv"), index=False)

#  EDA Plots

# 1. Sentiment Distribution
plt.figure(figsize=(6,4))
sns.countplot(x="classification", data=df_sent)
plt.title("Market Sentiment Distribution")
plt.savefig(os.path.join(out_dir, "sentiment_distribution.png"))
plt.close()

# 2. Closed PnL vs Sentiment
plt.figure(figsize=(8,5))
sns.boxplot(x="classification", y="closedpnl", data=df_merged)
plt.title("Closed PnL by Sentiment")
plt.savefig(os.path.join(out_dir, "pnl_vs_sentiment.png"))
plt.close()

# 3. Trade Value vs Sentiment (using Size USD if available)
if 'size_usd' in df_merged.columns:
    avg_trade_value = df_merged.groupby("classification")['size_usd'].mean().reset_index()
    plt.figure(figsize=(6,4))
    sns.barplot(x="classification", y="size_usd", data=avg_trade_value)
    plt.title("Average Trade Value (USD) by Sentiment")
    plt.savefig(os.path.join(out_dir, "trade_value_vs_sentiment.png"))
    plt.close()

# 4. Leverage vs Sentiment (only if leverage exists)
if 'leverage' in df_merged.columns:
    plt.figure(figsize=(8,5))
    sns.boxplot(x="classification", y="leverage", data=df_merged)
    plt.title("Leverage by Sentiment")
    plt.savefig(os.path.join(out_dir, "leverage_vs_sentiment.png"))
    plt.close()

# 📈 Win Rate Calculation
df_merged['win'] = df_merged['closedpnl'] > 0
win_rate = df_merged.groupby("classification")['win'].mean() * 100
win_rate_df = win_rate.reset_index()
win_rate_df.columns = ["classification", "win_rate(%)"]
win_rate_df.to_csv(os.path.join(csv_dir, "win_rate_by_sentiment.csv"), index=False)

print("\nSample of merged dataset:")
print(df_merged.head())



Sample of merged dataset:
                                      account  coin  execution_price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   

   size_tokens  size_usd side       timestamp_ist  start_position direction  \
0       986.87   7872.16  BUY 2024-12-02 22:50:00        0.000000       Buy   
1        16.00    127.68  BUY 2024-12-02 22:50:00      986.524596       Buy   
2       144.09   1150.63  BUY 2024-12-02 22:50:00     1002.518996       Buy   
3       142.98   1142.04  BUY 2024-12-02 22:50:00     1146.558564       Buy   
4         8.73     69.75  BUY 2024-12-02 22:50:00     1289.488521       Buy   

   closed_pnl                                   transaction_hash 